## Latent Dirichlet Allocation ##
**Watch these videos:**
- https://www.youtube.com/watch?v=azXCzI57Yfc
- https://www.youtube.com/watch?v=T05t-SqKArY
- https://www.youtube.com/watch?v=BaM1uiCpj_E

LDA is used to classify text in a document to a particular topic. It builds a topic per document model and words per topic model, modeled as Dirichlet distributions. 

* Each document is modeled as a multinomial distribution of topics and each topic is modeled as a multinomial distribution of words.
* LDA assumes that the every chunk of text we feed into it will contain words that are somehow related. Therefore choosing the right corpus of data is crucial. 
* It also assumes documents are produced from a mixture of topics. Those topics then generate words based on their probability distribution. 

## Step 1: Load the dataset

The dataset we'll use is a list of over one million news headlines published over a period of 15 years. We'll start by loading it from the `abcnews-date-text.csv` file.

In [1]:
'''
Load the dataset from the CSV and save it to 'data_text'. We only need the'headline_text' column,
so drop the rest of the columns. 
Then, add an 'index' column to the dataframe. Be sure to name it index.
Hint: dataframe.index
'''
import pandas as pd
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
data_text = data[:][['headline_text']];
data_text['index'] = data_text.index

documents = data_text

Let's glance at the dataset:

In [2]:
'''
print the total number of documents
'''
print(len(documents))

1103665


In [3]:
documents[:5]

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


## Step 2: Data Preprocessing ##

We will perform the following steps:

* **Tokenization**: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
* Words that have fewer than 3 characters are removed.
* All **stopwords** are removed.
* Words are **lemmatized** - words in third person are changed to first person and verbs in past and future tenses are changed into present.
* Words are **stemmed** - words are reduced to their root form.

This time, we'll make use of the gensim library and show you how  to preprocess text easily. 

In [4]:
'''
Loading Gensim and nltk libraries
'''
# pip install gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

In [5]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/rushil/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Lemmatizer Example
Before preprocessing our dataset, let's first look at an lemmatizing example. What would be the output if we lemmatized the word 'went':

In [6]:
print(WordNetLemmatizer().lemmatize('went', pos = 'v')) # past tense to present tense

go


### Stemmer Example
Let's also look at a stemming example. Let's throw a number of words at the stemmer and see how it deals with each one:

In [7]:
stemmer = SnowballStemmer("english")
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]

pd.DataFrame(data={'original word':original_words, 'stemmed':singles })

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [8]:
'''
Write a function to perform the pre processing steps on the entire dataset
'''
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            # TODO: Apply lemmatize_stemming on the token, then add to the results list
            result.append(lemmatize_stemming(token))
    return result



In [9]:
'''
Preview a document after preprocessing
'''
document_num = 4310
doc_sample = documents[documents['index'] == document_num].values[0][0]

print("Original document: ")
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))

Original document: 
['rain', 'helps', 'dampen', 'bushfires']


Tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


In [10]:
documents

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4
...,...,...
1103660,the ashes smiths warners near miss liven up bo...,1103660
1103661,timelapse: brisbanes new year fireworks,1103661
1103662,what 2017 meant to the kids of australia,1103662
1103663,what the papodopoulos meeting may mean for ausus,1103663


Let's now preprocess all the news headlines we have. To do that, let's use the [map](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.map.html) function from pandas to apply `preprocess()` to the `headline_text` column

**Note**: This may take a few minutes

In [11]:
# TODO: preprocess all the headlines, saving the list of results as 'processed_docs'
processed_docs = documents['headline_text'].map(preprocess)

In [12]:
'''
Preview 'processed_docs'
'''
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

## Step 3.1: Bag of words on the dataset

Now let's create a dictionary from 'processed_docs' containing the number of times a word appears in the training set. To do that, let's pass `processed_docs` to [`gensim.corpora.Dictionary()`](https://radimrehurek.com/gensim/corpora/dictionary.html) and call it '`dictionary`'.

In [13]:
'''
Create a dictionary from 'processed_docs' containing the number of times a word appears 
in the training set using gensim.corpora.Dictionary and call it 'dictionary'
'''
dictionary = gensim.corpora.Dictionary(processed_docs)

In [14]:
'''
Checking dictionary created
'''
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


** Gensim filter_extremes **

[`filter_extremes(no_below=5, no_above=0.5, keep_n=100000)`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.filter_extremes)

Filter out tokens that appear in

* less than no_below documents (absolute number) or
* more than no_above documents (fraction of total corpus size, not absolute number).
* after (1) and (2), keep only the first keep_n most frequent tokens (or keep all if None).

In [15]:
'''
OPTIONAL STEP
Remove very rare and very common words:

- words appearing less than 15 times
- words appearing in more than 10% of all documents
'''
# TODO: apply dictionary.filter_extremes() with the parameters mentioned above
dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n=100000)

** Gensim doc2bow **

[`doc2bow(document)`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.doc2bow)

* Convert document (a list of words) into the bag-of-words format = list of (token_id, token_count) 2-tuples. Each word is assumed to be a tokenized and normalized string (either unicode or utf8-encoded). No further preprocessing is done on the words in document; apply tokenization, stemming etc. before calling this method.

In [16]:
'''
Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear. Save this to 'bow_corpus'
'''
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [17]:
'''
Checking Bag of Words corpus for our sample document --> (token_id, token_count)
'''
bow_corpus[document_num]

[(76, 1), (112, 1), (483, 1), (4014, 1)]

In [18]:
'''
Preview BOW for our sample preprocessed document
'''
# Here document_num is document number 4310 which we have checked in Step 2
bow_doc_4310 = bow_corpus[document_num]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 76 ("bushfir") appears 1 time.
Word 112 ("help") appears 1 time.
Word 483 ("rain") appears 1 time.
Word 4014 ("dampen") appears 1 time.


## Step 3.2: TF-IDF on our document set ##

While performing TF-IDF on the corpus is not necessary for LDA implemention using the gensim model, it is recemmended. TF-IDF expects a bag-of-words (integer values) training corpus during initialization. During transformation, it will take a vector and return another vector of the same dimensionality.

*Please note: The author of Gensim dictates the standard procedure for LDA to be using the Bag of Words model.*

** TF-IDF stands for "Term Frequency, Inverse Document Frequency".**

* It is a way to score the importance of words (or "terms") in a document based on how frequently they appear across multiple documents.
* If a word appears frequently in a document, it's important. Give the word a high score. But if a word appears in many documents, it's not a unique identifier. Give the word a low score.
* Therefore, common words like "the" and "for", which appear in many documents, will be scaled down. Words that appear frequently in a single document will be scaled up.

In other words:

* TF(w) = `(Number of times term w appears in a document) / (Total number of terms in the document)`.
* IDF(w) = `log_e(Total number of documents / Number of documents with term w in it)`.

** For example **

* Consider a document containing `100` words wherein the word 'tiger' appears 3 times. 
* The term frequency (i.e., tf) for 'tiger' is then: 
    - `TF = (3 / 100) = 0.03`. 

* Now, assume we have `10 million` documents and the word 'tiger' appears in `1000` of these. Then, the inverse document frequency (i.e., idf) is calculated as:
    - `IDF = log(10,000,000 / 1,000) = 4`. 

* Thus, the Tf-idf weight is the product of these quantities: 
    - `TF-IDF = 0.03 * 4 = 0.12`.

In [19]:
'''
Create tf-idf model object using models.TfidfModel on 'bow_corpus' and save it to 'tfidf'
'''
from gensim import corpora, models
#tfidf = # TODO
tfidf = models.TfidfModel(bow_corpus)

In [20]:
'''
Apply transformation to the entire corpus and call it 'corpus_tfidf'
'''
#corpus_tfidf = # TODO
corpus_tfidf = tfidf[bow_corpus]

In [21]:
'''
Preview TF-IDF scores for our first document --> --> (token_id, tfidf score)
'''
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5892908644709983),
 (1, 0.38929657403503015),
 (2, 0.4964985198530063),
 (3, 0.5046520328695662)]


## Step 4.1: Running LDA using Bag of Words ##

We are going for 10 topics in the document corpus.

** We will be running LDA using all CPU cores to parallelize and speed up model training.**

Some of the parameters we will be tweaking are:

* **num_topics** is the number of requested latent topics to be extracted from the training corpus.
* **id2word** is a mapping from word ids (integers) to words (strings). It is used to determine the vocabulary size, as well as for debugging and topic printing.
* **workers** is the number of extra processes to use for parallelization. Uses all available cores by default.
* **alpha** and **eta** are hyperparameters that affect sparsity of the document-topic (theta) and topic-word (lambda) distributions. We will let these be the default values for now(default value is `1/num_topics`)
    - Alpha is the per document topic distribution.
        * High alpha: Every document has a mixture of all topics(documents appear similar to each other).
        * Low alpha: Every document has a mixture of very few topics

    - Eta is the per topic word distribution.
        * High eta: Each topic has a mixture of most words(topics appear similar to each other).
        * Low eta: Each topic has a mixture of few words.

* ** passes ** is the number of training passes through the corpus. For  example, if the training corpus has 50,000 documents, chunksize is  10,000, passes is 2, then online training is done in 10 updates: 
    * `#1 documents 0-9,999 `
    * `#2 documents 10,000-19,999 `
    * `#3 documents 20,000-29,999 `
    * `#4 documents 30,000-39,999 `
    * `#5 documents 40,000-49,999 `
    * `#6 documents 0-9,999 `
    * `#7 documents 10,000-19,999 `
    * `#8 documents 20,000-29,999 `
    * `#9 documents 30,000-39,999 `
    * `#10 documents 40,000-49,999` 

In [22]:
# LDA mono-core
# lda_model = gensim.models.LdaModel(bow_corpus, 
#                                    num_topics = 10, 
#                                    id2word = dictionary,                                    
#                                    passes = 50)

# LDA multicore 
'''
Train your lda model using gensim.models.LdaMulticore and save it to 'lda_model'
'''
# TODO
lda_model = gensim.models.LdaMulticore(bow_corpus, 
                                       num_topics=10, 
                                       id2word = dictionary, 
                                       passes = 2, 
                                       workers=2)

In [23]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic))
    print("\n")

Topic: 0 
Words: 0.033*"charg" + 0.026*"melbourn" + 0.024*"murder" + 0.018*"polic" + 0.018*"alleg" + 0.017*"famili" + 0.016*"accus" + 0.015*"tasmania" + 0.015*"drug" + 0.014*"child"


Topic: 1 
Words: 0.040*"polic" + 0.028*"death" + 0.022*"coast" + 0.018*"interview" + 0.018*"miss" + 0.017*"woman" + 0.014*"investig" + 0.014*"gold" + 0.012*"driver" + 0.012*"polit"


Topic: 2 
Words: 0.033*"queensland" + 0.016*"work" + 0.014*"take" + 0.014*"worker" + 0.012*"vote" + 0.011*"sydney" + 0.010*"warn" + 0.009*"strike" + 0.009*"western" + 0.009*"celebr"


Topic: 3 
Words: 0.034*"trump" + 0.030*"court" + 0.022*"govern" + 0.021*"year" + 0.020*"perth" + 0.019*"face" + 0.016*"hous" + 0.015*"donald" + 0.015*"tasmanian" + 0.014*"rise"


Topic: 4 
Words: 0.025*"south" + 0.025*"report" + 0.021*"australia" + 0.020*"test" + 0.018*"north" + 0.016*"say" + 0.015*"west" + 0.012*"guilti" + 0.011*"talk" + 0.011*"senat"


Topic: 5 
Words: 0.025*"kill" + 0.023*"crash" + 0.021*"die" + 0.017*"dead" + 0.014*"attack" 

### Classification of the topics ###

Using the words in each topic and their corresponding weights, what categories were you able to infer?

* 0: 
* 1: 
* 2: 
* 3: 
* 4: 
* 5: 
* 6: 
* 7:  
* 8: 
* 9: 

## Step 4.2 Running LDA using TF-IDF ##

In [24]:
'''
Define lda model using corpus_tfidf
'''
# TODO
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, 
                                             num_topics=10, 
                                             id2word = dictionary, 
                                             passes = 2, 
                                             workers=4)

In [25]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model_tfidf.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")

Topic: 0 Word: 0.009*"guilti" + 0.008*"abus" + 0.008*"plead" + 0.008*"kill" + 0.007*"juli" + 0.007*"friday" + 0.007*"polic" + 0.006*"decemb" + 0.006*"drive" + 0.006*"islam"


Topic: 1 Word: 0.017*"interview" + 0.013*"drum" + 0.007*"tuesday" + 0.007*"stori" + 0.007*"thursday" + 0.006*"andrew" + 0.006*"toni" + 0.006*"farm" + 0.005*"cancer" + 0.005*"obama"


Topic: 2 Word: 0.011*"charg" + 0.010*"jail" + 0.009*"assault" + 0.009*"alleg" + 0.009*"grandstand" + 0.009*"court" + 0.008*"sentenc" + 0.008*"murder" + 0.007*"child" + 0.007*"mother"


Topic: 3 Word: 0.020*"countri" + 0.019*"trump" + 0.019*"hour" + 0.010*"podcast" + 0.010*"turnbul" + 0.009*"donald" + 0.009*"weather" + 0.009*"tasmania" + 0.006*"violenc" + 0.006*"malcolm"


Topic: 4 Word: 0.014*"rural" + 0.011*"govern" + 0.009*"news" + 0.008*"fund" + 0.008*"health" + 0.007*"elect" + 0.007*"nation" + 0.006*"budget" + 0.006*"say" + 0.005*"plan"


Topic: 5 Word: 0.010*"woman" + 0.009*"polic" + 0.008*"coast" + 0.008*"murder" + 0.008*"charg"

### Classification of the topics ###

As we can see, when using tf-idf, heavier weights are given to words that are not as frequent which results in nouns being factored in. That makes it harder to figure out the categories as nouns can be hard to categorize. This goes to show that the models we apply depend on the type of corpus of text we are dealing with. 

Using the words in each topic and their corresponding weights, what categories could you find?

* 0: 
* 1:  
* 2: 
* 3: 
* 4:  
* 5: 
* 6: 
* 7: 
* 8: 
* 9: 

## Step 5.1: Performance evaluation by classifying sample document using LDA Bag of Words model##

We will check to see where our test document would be classified. 

In [26]:
'''
Text of sample document 4310
'''
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfir']

In [27]:
'''
Check which topic our test document belongs to using the LDA Bag of Words model.
'''

# Our test document is document number 4310
for index, score in sorted(lda_model[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.8199552893638611	 
Topic: 0.029*"elect" + 0.019*"countri" + 0.018*"state" + 0.017*"rural" + 0.015*"water" + 0.014*"peopl" + 0.013*"farmer" + 0.010*"govern" + 0.010*"victorian" + 0.010*"hobart"

Score: 0.020009418949484825	 
Topic: 0.033*"queensland" + 0.016*"work" + 0.014*"take" + 0.014*"worker" + 0.012*"vote" + 0.011*"sydney" + 0.010*"warn" + 0.009*"strike" + 0.009*"western" + 0.009*"celebr"

Score: 0.020005784928798676	 
Topic: 0.018*"chang" + 0.016*"council" + 0.015*"health" + 0.014*"fund" + 0.013*"hour" + 0.012*"indigen" + 0.012*"plan" + 0.012*"servic" + 0.011*"school" + 0.011*"children"

Score: 0.020005594938993454	 
Topic: 0.022*"adelaid" + 0.018*"market" + 0.016*"open" + 0.014*"home" + 0.014*"turnbul" + 0.013*"record" + 0.013*"leav" + 0.013*"fight" + 0.013*"share" + 0.012*"break"

Score: 0.020004380494356155	 
Topic: 0.033*"charg" + 0.026*"melbourn" + 0.024*"murder" + 0.018*"polic" + 0.018*"alleg" + 0.017*"famili" + 0.016*"accus" + 0.015*"tasmania" + 0.015*"drug" + 0.0

### It has the highest probability (`x`) to be  part of the topic that we assigned as X, which is the accurate classification. ###

## Step 5.2: Performance evaluation by classifying sample document using LDA TF-IDF model##

In [28]:
'''
Check which topic our test document belongs to using the LDA TF-IDF model.
'''
for index, score in sorted(lda_model_tfidf[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.8199507594108582	 
Topic: 0.020*"countri" + 0.019*"trump" + 0.019*"hour" + 0.010*"podcast" + 0.010*"turnbul" + 0.009*"donald" + 0.009*"weather" + 0.009*"tasmania" + 0.006*"violenc" + 0.006*"malcolm"

Score: 0.020006749778985977	 
Topic: 0.010*"woman" + 0.009*"polic" + 0.008*"coast" + 0.008*"murder" + 0.008*"charg" + 0.008*"price" + 0.008*"death" + 0.008*"north" + 0.007*"stab" + 0.006*"die"

Score: 0.020006736740469933	 
Topic: 0.014*"crash" + 0.012*"queensland" + 0.008*"polic" + 0.007*"shoot" + 0.007*"fatal" + 0.007*"wednesday" + 0.006*"east" + 0.006*"mark" + 0.006*"truck" + 0.006*"south"

Score: 0.0200064517557621	 
Topic: 0.014*"rural" + 0.011*"govern" + 0.009*"news" + 0.008*"fund" + 0.008*"health" + 0.007*"elect" + 0.007*"nation" + 0.006*"budget" + 0.006*"say" + 0.005*"plan"

Score: 0.020005835220217705	 
Topic: 0.009*"royal" + 0.008*"australia" + 0.008*"commiss" + 0.007*"world" + 0.007*"rugbi" + 0.007*"octob" + 0.007*"leagu" + 0.007*"live" + 0.006*"cricket" + 0.005*"final

### It has the highest probability (`x%`) to be  part of the topic that we assigned as X. ###

## Step 6: Testing model on unseen document ##

In [29]:
unseen_document = "My favorite sports activities are running and swimming."

# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.4200761020183563	 Topic: 0.033*"charg" + 0.026*"melbourn" + 0.024*"murder" + 0.018*"polic" + 0.018*"alleg"
Score: 0.21999646723270416	 Topic: 0.020*"live" + 0.019*"china" + 0.019*"hospit" + 0.016*"nation" + 0.015*"world"
Score: 0.21989253163337708	 Topic: 0.018*"chang" + 0.016*"council" + 0.015*"health" + 0.014*"fund" + 0.013*"hour"
Score: 0.020004983991384506	 Topic: 0.040*"polic" + 0.028*"death" + 0.022*"coast" + 0.018*"interview" + 0.018*"miss"
Score: 0.020004983991384506	 Topic: 0.033*"queensland" + 0.016*"work" + 0.014*"take" + 0.014*"worker" + 0.012*"vote"
Score: 0.020004983991384506	 Topic: 0.034*"trump" + 0.030*"court" + 0.022*"govern" + 0.021*"year" + 0.020*"perth"
Score: 0.020004983991384506	 Topic: 0.025*"south" + 0.025*"report" + 0.021*"australia" + 0.020*"test" + 0.018*"north"
Score: 0.020004983991384506	 Topic: 0.025*"kill" + 0.023*"crash" + 0.021*"die" + 0.017*"dead" + 0.014*"attack"
Score: 0.020004983991384506	 Topic: 0.022*"adelaid" + 0.018*"market" + 0.016*"o

The model correctly classifies the unseen document with 'x'% probability to the X category.

### Now, look up different ways we can transform our words or use different techniques or improvements that we can make to make our Topic Classifier more accurate. Write them down below:

**We can:**
- Append points here
- 